In [33]:
from gurobipy import Model, GRB, quicksum

In [34]:
# Define the model
m = Model("DonutPriceOptimization")

In [35]:
# Given data
cost_jelly = 0.46 * 2  # Cost for a 2-pack of jelly-filled donuts
cost_regular = 0.33 * 2  # Cost for a 2-pack of regular donuts
base_price_jelly = 3.49  # Base price for jelly-filled donuts
base_price_regular = 2.99  # Base price for regular donuts
max_demand_jelly = [66, 76, 116, 86, 56, 46]  # Max demand for jelly-filled donuts
max_demand_regular = [72, 70, 123, 93, 51, 41]  # Max demand for regular donuts
slope_jelly = [14, 13, 11, 8, 24, 28]  # Slope for jelly-filled donuts
slope_regular = [15, 12, 13, 8, 20, 42]  # Slope for regular donuts

In [36]:
# Define decision variables
pn = m.addVars(6, lb=0.0, vtype=GRB.CONTINUOUS, name="pn")  # Prices for jelly-filled donuts
qn = m.addVars(6, lb=0.0, vtype=GRB.CONTINUOUS, name="qn")  # Prices for regular donuts

In [37]:
# Define the objective function
m.setObjective(
    quicksum((pn[n] - cost_jelly) * (max_demand_jelly[n] - slope_jelly[n] * (pn[n] - base_price_jelly)) for n in range(6)) +
    quicksum((qn[n] - cost_regular) * (max_demand_regular[n] - slope_regular[n] * (qn[n] - base_price_regular)) for n in range(6)),
    GRB.MAXIMIZE
)

In [38]:
# Define constraints

# Constraint for the price to increase until 8pm and then decrease
for n in range(1, 3):
    m.addConstr(pn[n] >= pn[n-1], f"pn_increase_{n}")
    m.addConstr(qn[n] >= qn[n-1], f"qn_increase_{n}")
for n in range(3, 5):
    m.addConstr(pn[n] <= pn[n+1], f"pn_decrease_{n}")
    m.addConstr(qn[n] <= qn[n+1], f"qn_decrease_{n}")

# Constraint to ensure we do not sell more than we have in stock
m.addConstr(quicksum(max_demand_jelly[n] - slope_jelly[n] * (pn[n] - base_price_jelly) for n in range(6)) <= 274, "Stock_Jelly")
m.addConstr(quicksum(max_demand_regular[n] - slope_regular[n] * (qn[n] - base_price_regular) for n in range(6)) <= 222, "Stock_Regular")

# Constraint to ensure we do not exceed the maximum market demand at each hour
for n in range(6):
    m.addConstr(max_demand_jelly[n] - slope_jelly[n] * (pn[n] - base_price_jelly) >= 0, f"Demand_Jelly_{n}")
    m.addConstr(max_demand_regular[n] - slope_regular[n] * (qn[n] - base_price_regular) >= 0, f"Demand_Regular_{n}")

In [39]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 12 columns and 40 nonzeros
Model fingerprint: 0x2a65f885
Model has 12 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+02, 2e+02]
  QObjective range [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 6e+02]
Presolve removed 12 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 12 columns, 28 nonzeros
Presolved model has 12 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.200e+01
 Factor NZ  : 3.000e+01
 Factor Ops : 1.100e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.24493297e+08  2.31675772e+08  9.98e+02 9.4

In [40]:
# Print the results
if m.status == GRB.OPTIMAL:
    jelly_prices = m.getAttr('x', pn)
    regular_prices = m.getAttr('x', qn)
    print("Optimal prices for jelly-filled donut packs:")
    for n in range(6):
        print(f"Hour {n+6} PM: ${jelly_prices[n]:.2f}")
    print("\nOptimal prices for regular donut packs:")
    for n in range(6):
        print(f"Hour {n+6} PM: ${regular_prices[n]:.2f}")
else:
    print("Optimal solution was not found.")

Optimal prices for jelly-filled donut packs:
Hour 6 PM: $5.33
Hour 7 PM: $5.89
Hour 8 PM: $8.24
Hour 9 PM: $4.54
Hour 10 PM: $4.54
Hour 11 PM: $4.54

Optimal prices for regular donut packs:
Hour 6 PM: $6.07
Hour 7 PM: $6.59
Hour 8 PM: $8.40
Hour 9 PM: $3.97
Hour 10 PM: $3.97
Hour 11 PM: $3.97


In [41]:
from gurobipy import *

# Create a new model
m = Model("DonutPriceOptimization")

# Decision variables
# pn = price for jelly-filled 2-donut packs
# qn = price for regular 2-donut packs
pn = m.addVars(6, lb=0.0, name="pn")
qn = m.addVars(6, lb=0.0, name="qn")

# Parameters (example values, replace with your data)
cost_jelly = 0.46 * 2  # Cost for a 2-pack of jelly-filled donuts
cost_regular = 0.33 * 2  # Cost for a 2-pack of regular donuts
max_demand_jelly = [66, 76, 116, 86, 56, 46]  # Max demand for jelly-filled donuts
max_demand_regular = [72, 70, 123, 93, 51, 41]  # Max demand for regular donuts
slope_jelly = [14, 13, 11, 8, 24, 28]  # Slope for jelly-filled donuts
slope_regular = [15, 12, 13, 8, 20, 42]  # Slope for regular donuts

# Demand function
def dJ(price, n):
    # Demand for jelly-filled donuts
    return max_demand_jelly[n] - slope_jelly[n] * (price - 3.49)

def dR(price, n):
    # Demand for regular donuts
    return max_demand_regular[n] - slope_regular[n] * (price - 2.99)

# Objective function
m.setObjective(
    quicksum((pn[n] - cost_jelly) * dJ(pn[n], n) for n in range(6)) +
    quicksum((qn[n] - cost_regular) * dR(qn[n], n) for n in range(6)),
    GRB.MAXIMIZE
)

# Constraints
# Total demand constraints
m.addConstr(quicksum(dJ(pn[n], n) for n in range(6)) <= 137, "TotalJellyDemand")
m.addConstr(quicksum(dR(qn[n], n) for n in range(6)) <= 111, "TotalRegularDemand")

# Price increase and decrease constraints
m.addConstrs((pn[n] <= pn[n+1] for n in range(1)), "JellyPriceIncrease")
m.addConstrs((pn[n] >= pn[n+1] for n in range(3, 5)), "JellyPriceDecrease")
m.addConstrs((qn[n] <= qn[n+1] for n in range(1)), "RegularPriceIncrease")
m.addConstrs((qn[n] >= qn[n+1] for n in range(3, 5)), "RegularPriceDecrease")

# The price of jelly must be at least 5% higher than regular
m.addConstrs((pn[n] >= 1.05 * qn[n] for n in range(6)), "PriceDifference")

# Non-negativity constraints for decision variables
m.addConstrs((pn[n] >= 0 for n in range(6)), "NonNegativityJelly")
m.addConstrs((qn[n] >= 0 for n in range(6)), "NonNegativityRegular")

# Optimize the model
m.optimize()

# Print the results
if m.status == GRB.OPTIMAL:
    jelly_prices = m.getAttr('x', pn)
    regular_prices = m.getAttr('x', qn)
    print("Optimal prices for jelly-filled donut packs:")
    for n in range(6):
        print(f"Hour {n+6} PM: ${jelly_prices[n]:.2f}")
    print("\nOptimal prices for regular donut packs:")
    for n in range(6):
        print(f"Hour {n+6} PM: ${regular_prices[n]:.2f}")
else:
    print("Optimal solution was not found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 12 columns and 48 nonzeros
Model fingerprint: 0xef2ac5d0
Model has 12 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+02, 2e+02]
  QObjective range [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+02, 7e+02]
Presolve removed 12 rows and 0 columns
Presolve time: 0.00s
Presolved: 14 rows, 12 columns, 36 nonzeros
Presolved model has 12 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.200e+01
 Factor NZ  : 1.050e+02
 Factor Ops : 1.015e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.24346534e+08  2.26227015e+08  5.03e+01 0.0